In [ ]:
import torch
import torch.nn as nn
import math
from torch.nn.parameter import Parameter
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
device = 'cuda:0'
path = '/home/zhangjx/All_model/genration_scene/VL-T5/t5-base/pytorch_model.bin'
weight = torch.load(path)['encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight']
is_decoder = False
relative_attention_num_buckets = 32
relative_attention_max_distance = 128
relative_attention_bias = nn.Embedding(32,12)
relative_attention_bias.weight = Parameter(weight)

In [ ]:
def _relative_position_bucket(relative_position, bidirectional=True, num_buckets=32, max_distance=128):
    relative_buckets = 0
    if bidirectional:
        num_buckets //= 2
        relative_buckets += (relative_position > 0).to(torch.long) * num_buckets # 这个是做什么的
        relative_position = torch.abs(relative_position)
    else:
        relative_position = -torch.min(relative_position, torch.zeros_like(relative_position))
    # print(relative_buckets)
    # now relative_position is in the range [0, inf)

    # half of the buckets are for exact increments in positions
    max_exact = num_buckets // 2
    is_small = relative_position < max_exact
    # print(is_small)
    # The other half of the buckets are for logarithmically bigger bins in positions up to max_distance
    relative_position_if_large = max_exact + (
        torch.log(relative_position.float() / max_exact)
        / math.log(max_distance / max_exact)
        * (num_buckets - max_exact)
    ).to(torch.long)
    # print(relative_position_if_large)
    relative_position_if_large = torch.min(
        relative_position_if_large, torch.full_like(relative_position_if_large, num_buckets - 1)
    ) # 截断
    # tmp = relative_position_if_large
    # tmp[tmp<0]=0
    # print(tmp)
    # print(is_small)
    relative_buckets += torch.where(is_small, relative_position, relative_position_if_large) # 8个以内取原值，8个以外取大值
    return relative_buckets

def compute_bias(query_length, key_length, device=None):
    """Compute binned relative position bias"""
    if device is None:
        device = relative_attention_bias.weight.device
    context_position = torch.arange(query_length, dtype=torch.long, device=device)[:, None]
    memory_position = torch.arange(key_length, dtype=torch.long, device=device)[None, :]
    relative_position = memory_position - context_position  # shape (query_length, key_length)
    relative_position_bucket = _relative_position_bucket(
        relative_position,  # shape (query_length, key_length)
        bidirectional=(not is_decoder),
        num_buckets=relative_attention_num_buckets,
        max_distance=relative_attention_max_distance,
    )
    print(relative_position_bucket)
    values = relative_attention_bias(relative_position_bucket)  # shape (query_length, key_length, num_heads)
    values = values.permute([2, 0, 1]).unsqueeze(0)  # shape (1, num_heads, query_length, key_length)
    return values

In [ ]:
position_bias = compute_bias(100,100)

In [ ]:
plt.imshow(position_bias[0,3].detach().numpy())

In [ ]:
def getPositionEncoding(seq_len, d, n=10000):
    P = np.zeros((seq_len, d))
    for t in range(seq_len):
        for k in np.arange(int(d/2)):
            denominator = np.power(n, 2*k/d)
            P[t, 2*k] = np.sin(t/denominator)
            P[t, 2*k+1] = np.cos(t/denominator)
    return P

In [ ]:
P = getPositionEncoding(seq_len=50, d=128, n=50)
cax = plt.matshow(P,cmap=plt.cm.Blues)
plt.gcf().colorbar(cax)